<center><h1>Table Wrangling</h1></center>

## Preliminaries

Where we are heading is toward making predictions from the Titanic data. The critical prediction is on survival. We would like to use information about a passenger (gender, age, cabin class) to predict whether they survived the sinking. The general idea is that we will build our own predictor, try it on a passenger to get a guess (survived, perished) and then check our guess against the historical record to see if we were right. The better guesses we make, the more accurate is our predictor.

## Predicting Survival

![Sinking of the RMS Titanic](https://pbs.twimg.com/profile_images/1305956322/titanic2.jpg "Sinking of the RMS Titanic")

We will be working with a the passenger information (rows) from the Titanic sinking. Each row in our table has values for 11 fields describing that individual passenger. Our goal is to use one or more of 10 fields to predict the 11th field, `Survived`. Here is what we have to work with.

Let's load the data into a python table (same as with last module).

In [19]:
from csv import DictReader # helps with handling csv formatted data

from urllib2 import urlopen # helps with pulling data off the web

url = 'https://docs.google.com/spreadsheets/d/1z1ycUZjJpmMWB4gXbhwRQ9B_qa42CwzAQkf82mLibxI/pub?output=csv'

response = urlopen(url)

titanic_table = [row for row in DictReader(response)]
print(titanic_table[0])  # print the first row of the table - Mr. Braund

{'Fare': '7.25', 'Name': 'Braund, Mr. Owen Harris', 'Embarked': 'S', 'Age': '22', 'Parch': '0', 'Pclass': '3', 'Sex': 'male', 'Survived': '0', 'SibSp': '1', 'PassengerId': '1', 'Ticket': 'A/5 21171', 'Cabin': ''}


What we have is a table represented as a list of dictionaries, no different than we have been working with in previous modules. Let's take the first row and print each of the 11 values.

In [20]:
row1 = titanic_table[0]  # remember lists start at index 0
print(row1['Name'])
print(row1['Age'])
print(row1['Sex'])
print(row1['Pclass'])
print(row1['Fare'])
print(row1['Parch'])
print(row1['SibSp'])
print(row1['Cabin'])  # notice that this prints a blank line
print(row1['Ticket'])
print(row1['PassengerId'])
print(row1['Survived'])  # what we will want to eventually predict

Braund, Mr. Owen Harris
22
male
3
7.25
0
1

A/5 21171
1
0


Notice that the value for `Cabin` is a blank line. This means that the `Cabin` value for Mr. Braund is missing; could not be found in the historical record. This is typical of spreadsheet data - we often have missing cells/values and will have to figure out what to do about it.

## Everything is a string

The `print` function will print "3" and 3 the same so we don't know if the value for Pclass is an integer or a string. I can tell you at this point, every value is a string. That's just the way the table is constructed from an outside file. Let's verify this using a new function called `type`. The `type` function takes a variable and tells you what data type it is. BTW: `str` stands for string.

In [21]:
type(row1['Pclass'])  # str

str

We will have to keep this in mind as we explore the data further. Initially, all values are strings even if they look like they should be numbers.

## How many survived?

We answered this question at the end of the last module but let's repeat it here.

In [22]:
survived_column = [row['Survived'] for row in titanic_table]  # string values
int_survived_column = [int(val) for val in survived_column]  # int values
sum_survivors = sum(int_survived_column)  # reduce to get total

print(sum_survivors)  # not many!

342


Now let's compute the percentage.

In [23]:
survival_percentage = float(sum_survivors)/len(titanic_table)  # float to avoid truncation
print(survival_percentage)  # 38% is not very good!

0.383838383838


Those two maps we just did? They would be considered "data wrangling". General idea is that the raw data you get, such as that in `titanic_table`, is rarely in perfect shape to start analyzing. So we will have to massage it (wrangle it) into form. Mapping a column from strings to ints would be part of the process. We will look at another data wrangling operation next.

## What about missing values?

First, let's do a filtered map of all passengers who are missing their age value. Remember a filtered map is like a normal map but adds a 3rd piece to the very right, the filter. We are filtering on the value being the empty string. Empty cell values in a csv file are represented by the empty string in Python. So we are asking for all passengers whose Age value is empty, i.e. not recorded.

In [24]:
missing_ages = [row['Age'] for row in titanic_table if row['Age'] == ""]
print( len(missing_ages))  # 177

177


Wow. That's on the edge of being too many missing Age values. If we want to use the Age column in our predictor, we are stuck if no Age value to work with. So we should consider wrangling that column to fill in missing values. What to do about missing values is a really interesting problem. Options we might consider here:

1. Use the mean of the column to fill in missing values.
2. Use the median of the column to fill in missing values.
3. Use the mode of the column to fill in missing values.
4. More advanced, use other column values to fill in missing values.
5. Don't fill in missing values - a missing value might be a predictor itself.

The fancy word for above is *impute*: guess a value based on other info. There is no one right answer. The question is do any of the above improve predictive power? The first 3 are relatively easy to explore. You can compute the mean, median and mode of the Age column once and then try each out in the predictor. The 4th option is worth considering. For instance, perhaps we can guess an age by looking at the corresponding fields of `Sibsp` and `Parch`, which have to do with parents and children. Getting to an actual value will be a journey but might be worth it. We would be giving every passenger with a missing age a value based on their personal attributes, not some global value like mean. The 5th option is also interesting and we will explore it further in coming modules.

Just to reiterate, we don't know what the real ages are for the missing values. So we cannot guess then see how we did. The critical question is do the values we fill in for the missing Ages help with overall prediction? If I fill in missing values in the Age column with the mean, does it give me higher prediction accuracy? We will look at this in more detail in the next module.

## Mean

In case we need it later, let's compute the mean for the Age column. I am hoping you might have come up with these steps on your own for mean:
1. Do a filtered map to get the non-empty age values.
2. Do a map to change them from strings to floats. We need float because some ages are given as fractions, e.g., 28.5.
3. Do a reduce using `sum`.
4. Calculate mean.

In [25]:
age_column = [row['Age'] for row in titanic_table if row['Age'] != ""]
float_age_column = [float(val) for val in age_column]
age_sum = sum(float_age_column)
age_mean = age_sum/len(float_age_column)  # sum/n
print(age_mean)  # 30.0 with round

29.6991176471


## What about missing gender values?

Fortunately, there are no missing gender values in the Titanic data. But you may not always be so lucky with your own datasets. Gender is a binary value as is Survival. It does not make sense to take the mean of binary columns. If you find you have missing values in a binary column, think about using the mode as default - the value that appears most often. The assessment that goes with this module explores this idea in more detail with the loan dataset.

## Next up

We looked at some beginning ideas in data wrangling in this module. The next module will be a start on doing prediction.